#Feature Extractions

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Specify the name of the uploaded file (ensure this is the exact name as it appears in Colab)
image_name = 'fe1.png'  # Replace with the actual image file name

# Load the image
image = cv2.imread(image_name, cv2.IMREAD_COLOR)

# Check if the image was loaded successfully
if image is None:
    print(f"Error: Unable to load the image '{image_name}'. Please check if the file exists in the correct directory.")
else:
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize the MSER detector with a higher delta for fewer regions
    mser = cv2.MSER_create(delta=20)

    # Detect MSER regions
    regions, _ = mser.detectRegions(gray_image)

    # Create a copy of the original image to draw the regions on
    image_with_regions = image.copy()

    # Draw MSER regions on the image
    for region in regions:
        x, y, w, h = cv2.boundingRect(region)
        cv2.rectangle(image_with_regions, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the image with MSER regions highlighted
    cv2_imshow(image_with_regions)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_image(image_path, gray=True):
    """ Load the input aerial image. """
    img = cv2.imread(image_path)
    if gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def extract_brief_features(image, max_keypoints=500):
    """
    Extract BRIEF descriptors from aerial image.
    :param image: Grayscale input image.
    :param max_keypoints: Max number of keypoints to detect using STAR detector.
    :return: Keypoints and BRIEF descriptors.
    """
    # Detect keypoints using FAST or STAR
    keypoint_detector = cv2.xfeatures2d.StarDetector_create()
    keypoints = keypoint_detector.detect(image)

    # Sort and keep the top N keypoints (for consistency)
    keypoints = sorted(keypoints, key=lambda x: -x.response)[:max_keypoints]

    # Apply BRIEF
    brief_extractor = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    keypoints, descriptors = brief_extractor.compute(image, keypoints)

    return keypoints, descriptors

def visualize_keypoints(image, keypoints, title="BRIEF Keypoints"):
    """ Visualize detected keypoints on the image. """
    img_with_kp = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0),
                                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(10, 6))
    plt.imshow(img_with_kp, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def brief_descriptor_to_binary(descriptor):
    """
    Convert a BRIEF descriptor to a binary string representation (as per Equation 10).
    """
    return ''.join(['1' if val else '0' for val in descriptor.astype(bool)])

def show_sample_descriptor(descriptors, index=0):
    """
    Print the binary descriptor of a selected keypoint.
    """
    if descriptors is not None and index < len(descriptors):
        binary_string = brief_descriptor_to_binary(descriptors[index])
        print(f"Binary BRIEF descriptor for keypoint {index}:\n{binary_string}")
    else:
        print("Descriptor not available.")

# ========================= RUNNING SECTION =========================

if __name__ == "__main__":
    image_path = "aerial_image.jpg"  # Replace with your actual path

    # Load image
    gray_img = load_image(image_path)

    # Extract BRIEF features
    keypoints, descriptors = extract_brief_features(gray_img)

    # Visualize the keypoints
    visualize_keypoints(gray_img, keypoints)

    # Show sample descriptor
    show_sample_descriptor(descriptors, index=0)


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def affine_transform(image, angle, scale=1.0):
    """
    Apply affine transformation to the image.

    Args:
    image (ndarray): Input image.
    angle (float): Angle for rotation.
    scale (float): Scaling factor.

    Returns:
    transformed_image: Affine-transformed image.
    """
    rows, cols = image.shape[:2]

    # Create rotation matrix
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)

    # Apply affine transformation
    transformed_image = cv2.warpAffine(image, M, (cols, rows))

    return transformed_image

def extract_sift_features(image):
    """
    Function to extract SIFT features from the image.

    Args:
    image (ndarray): Input image.

    Returns:
    keypoints, descriptors: Detected keypoints and computed descriptors.
    """
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)

    return keypoints, descriptors

def draw_keypoints(image, keypoints, max_keypoints=500):
    """
    Draw keypoints on the image, showing only the strongest ones.

    Args:
    image (ndarray): Input image.
    keypoints (list): Detected keypoints.
    max_keypoints (int): Maximum number of strongest keypoints to draw.

    Returns:
    Image with keypoints drawn.
    """
    # Sort keypoints by their response (strength) and keep the top N keypoints
    keypoints = sorted(keypoints, key=lambda x: x.response, reverse=True)[:max_keypoints]

    # Draw keypoints with smaller circles for cleaner visualization
    output_image = cv2.drawKeypoints(image, keypoints, None,
                                     flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
                                     color=(10, 0, 255))
    return output_image

# Load your aerial image (replace with your own image name)
image = cv2.imread('fe1.png')

if image is None:
    print("Error: Unable to load the image.")
else:
    # Preprocess the image (optional)
    # Apply Gaussian blur to reduce noise
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # List of angles to simulate different affine transformations
    angles = [0, 15, 30, 45, 60, 75, 90]

    all_keypoints = []
    all_descriptors = []

    # Apply different affine transformations
    for angle in angles:
        transformed_image = affine_transform(image, angle)

        # Extract SIFT features from the transformed image
        keypoints, descriptors = extract_sift_features(transformed_image)

        all_keypoints.extend(keypoints)
        if descriptors is not None:
            all_descriptors.append(descriptors)

    # Combine all descriptors
    if all_descriptors:
        all_descriptors = np.vstack(all_descriptors)

    # Draw only the strongest keypoints on the original image
    image_with_keypoints = draw_keypoints(image, all_keypoints, max_keypoints=5000)

    # Display the result
    cv2_imshow(image_with_keypoints)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f"Total number of keypoints detected (before filtering): {len(all_keypoints)}")
